In [38]:
%load_ext autoreload
%autoreload 2
from notebook import *
# if get something about NUMEXPR_MAX_THREADS being set incorrectly, don't worry.  It's not a problem.

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Multithreaded Architectures

Multithreaded processors are everywhere!!! Take a look of our processor again!

In [2]:
! lscpu

Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          46 bits physical, 48 bits virtual
  Byte Order:             Little Endian
CPU(s):                   24
  On-line CPU(s) list:    0-23
Vendor ID:                GenuineIntel
  Model name:             13th Gen Intel(R) Core(TM) i7-13700
    CPU family:           6
    Model:                183
    Thread(s) per core:   2
    Core(s) per socket:   16
    Socket(s):            1
    Stepping:             1
    CPU(s) scaling MHz:   38%
    CPU max MHz:          5200.0000
    CPU min MHz:          800.0000
    BogoMIPS:             4224.00
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge m
                          ca cmov pat pse36 clflush dts acpi mmx fxsr sse sse2 s
                          s ht tm pbe syscall nx pdpe1gb rdtscp lm constant_tsc 
                          art arch_perfmon pebs bts rep_good nopl xtopology nons
                          top_tsc c

## Share memory

In [3]:
compare([do_render_code("testloop.c", show="main"), do_render_code("testloop.c", show=["#else","#endif"])])

In [39]:
! make clean; make testloop; make testloop_O3

rm -f testloop testloop_O3 coherence blockmm blockmm_pthread value_of_i vadd_sse fence mfence vadd_sse_two_threads vadd_sse_two_threads_chunk vadd_mt vadd_mt_chunk testloop_volatile testloop_O3_volatile coherence_lock
rm -rf *.dSYM
gcc -O0 -g testloop.c -pthread -lpthread -o testloop 
gcc  -O3 -g testloop.c -pthread -lpthread -o testloop_O3
testloop.c: In function ‘modifyloop’:
testloop.c:23:3: warning: ignoring return value of ‘scanf’ declared with attribute ‘warn_unused_result’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wunused-result-Wunused-result]8;;]
   23 |   scanf("%d",&loop);
      |   ^~~~~~~~~~~~~~~~~


In [40]:
! echo 100 | ./testloop 

Please input a number:
Finished. User input is: 100


In [ ]:
! echo 100 | ./testloop_O3 

Please input a number:


In [6]:
compare([do_render_code("testloop.c", show="main"), do_render_code("testloop.c", show=["#ifdef","#else"])])

In [7]:
! make clean; make testloop_volatile; make testloop_O3_volatile

rm -f testloop testloop_O3 coherence blockmm blockmm_pthread value_of_i vadd_sse fence mfence vadd_sse_two_threads vadd_sse_two_threads_chunk vadd_mt vadd_mt_chunk testloop_volatile testloop_O3_volatile coherence_lock
rm -rf *.dSYM
gcc -O0 -g testloop.c -pthread -lpthread -o testloop_volatile 
gcc  -O3 -DVOLATILE testloop.c -pthread -lpthread -o testloop_O3_volatile
testloop.c: In function ‘modifyloop’:
testloop.c:13:3: warning: ignoring return value of ‘scanf’ declared with attribute ‘warn_unused_result’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wunused-result-Wunused-result]8;;]
   13 |   scanf("%d",&loop);
      |   ^~~~~~~~~~~~~~~~~


In [11]:
! echo 10 | ./testloop_volatile

Please input a number:
Finished. User input is: 10


In [8]:
! echo 10 | ./testloop_O3_volatile

Please input a number:
Finished. User input is: 10


## Coherency 

Coherency is the protocol maintaining what values should be seen in the system.

Can you read the following code and guess what's the output?

In [9]:
render_code("coherence.c")

// coherence.c:1-33 (33 lines)
#include <stdio.h>
#include <stdlib.h>
#include <pthread.h>
#include <unistd.h>

volatile int loop;

void* modifyloop(void *x)
{
//  sleep(1);

  while(loop < 1000)
  {
    loop++;
  }
  return NULL;
}

int main()
{
  pthread_t thread;
  loop = 1;
  
  pthread_create(&thread, NULL, modifyloop, NULL);
  while(loop<1000)
  {
    fprintf(stderr,"%d ", loop);
  }
  pthread_join(thread, NULL);
  
  fprintf(stderr,"%4d finished\n", loop);
  return 0;
}

Let's run it several times and see what do we get!

In [10]:
! make coherence; ./coherence >& coherence.out; tail -c 1000 coherence.out

gcc -O3 coherence.c -pthread -lpthread -o coherence 
1 1 1 984 1000 finished


In [11]:
for x in range(1, 20):
    !./coherence >& coherence.out; tail -c 1000 coherence.out


1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 556 1000 finished
1 1 1 1 1 1 1 1 1 1 1 1 1 1 710 1000 finished
1 571 1000 finished
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 185 1000 finished
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 513 1000 finished
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 363 1000 finished
1 1 1 1 1 1 1000 finished
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 190 1000 finished
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 210 1000 finished
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 675 1000 finished
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1000 finished
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 435 1000 finished
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 239 890 1000 finished
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 10

### Even with locks...

### Share memory example

In [12]:
compare([do_render_code("vadd_mt.c",show=["// interleaving","#endif"]),do_render_code("vadd_mt.c",show=["CHUNK","#else"])])

In [15]:
! make clean; make vadd_mt_chunk; make vadd_mt

rm -f testloop testloop_O3 coherence blockmm blockmm_pthread value_of_i vadd_sse fence mfence vadd_sse_two_threads vadd_sse_two_threads_chunk vadd_mt vadd_mt_chunk testloop_volatile testloop_O3_volatile coherence_lock
rm -rf *.dSYM
cc -O3 -Wno-format-zero-length -Wno-implicit-function-declaration -mno-avx -DHAVE_LINUX_PERF_EVENT_H -pthread -DSSE -DCHUNK vadd_mt.c perfstats.c -o vadd_mt_chunk 
cc -O3 -Wno-format-zero-length -Wno-implicit-function-declaration -mno-avx -DHAVE_LINUX_PERF_EVENT_H -pthread -DSSE vadd_mt.c perfstats.c -o vadd_mt


In [18]:
! echo "size,threads,IC,Cycles,CPI,CT,ET,L1_dcache_miss_rate,L1_dcache_misses,L1_dcache_accesses,branches,branch_misses" > stats.csv
! echo -n "16777216,4," >> stats.csv
! echo "interleaving:"; cd ~/courses/CS203/demo/multiprocessor; perf stat -e cycles,instructions,cache-misses ./vadd_mt
! echo -n "16777216,4," >> stats.csv
! echo "chunking:"; cd ~/courses/CS203/demo/multiprocessor; perf stat -e cycles,instructions,cache-misses  ./vadd_mt_chunk
display_df_mono(render_csv("stats.csv"))

interleaving:
67108864	67108864	67108864	67108864	Takes 0.257449 seconds
132.000000	

 Performance counter stats for './vadd_mt':

    25,906,327,675      cpu_atom/cycles/                                                        (1.05%)
    38,058,034,708      cpu_core/cycles/                                                        (97.80%)
    17,042,274,682      cpu_atom/instructions/           #    0.66  insn per cycle              (1.05%)
    48,702,256,536      cpu_core/instructions/           #    1.88  insn per cycle              (97.82%)
       103,248,281      cpu_atom/cache-misses/                                                  (1.05%)
       141,058,734      cpu_core/cache-misses/                                                  (97.83%)

       6.773625960 seconds time elapsed

       6.674226000 seconds user
       0.831654000 seconds sys


chunking:
67108864	67108864	67108864	67108864	Takes 0.094404 seconds
132.000000	

 Performance counter stats for './vadd_mt_chunk':

  

,index,size,threads,IC,Cycles,CPI,CT,ET,L1_dcache_miss_rate,L1_dcache_misses,L1_dcache_accesses,branches,branch_misses
0,0,16777216,4,1715279966,4283832712,2.497454,0.059715,0.255808,0.102211,74726746,731101688,246924162,83029
1,1,16777216,4,431807248,1535002603,3.554833,0.060505,0.092876,0.085149,15589326,183083252,61911888,23860


## Consistency

In [53]:
render_code("fence.c", show=["#ifndef MFENCE","endif"])

// fence.c:13-28 (16 lines)
#ifndef MFENCE
void* modifya(void *z)
{
  while(!go);
  a=1;
  x=b;
  return NULL;
}
void* modifyb(void *z)
{
  while(!go);
  b=1;
  y=a;
  return NULL;
}
#endif

In [54]:
! make clean; make fence

rm -f testloop testloop_O3 coherence blockmm blockmm_pthread value_of_i vadd_sse fence mfence vadd_sse_two_threads vadd_sse_two_threads_chunk vadd_mt vadd_mt_chunk testloop_volatile testloop_O3_volatile coherence_lock
rm -rf *.dSYM
cc -O3 -Wno-format-zero-length -Wno-implicit-function-declaration fence.c -o fence -lpthread


In [55]:
!rm fence.txt;  touch fence.txt
!lscpu | grep "Model name:"
!cd ~/courses/CS203/demo/multiprocessor; for i in {1..5000}; do ./fence 2>> fence.txt; done; for i in {1..5000}; do ./fence 2>> fence.txt; done;

Model name:                           13th Gen Intel(R) Core(TM) i7-13700


In [56]:
! grep "(0, 0)" fence.txt |wc
! grep "(0, 1)" fence.txt |wc
! grep "(1, 1)" fence.txt |wc
! grep "(1, 0)" fence.txt |wc

      5      20      95
   9332   37328  177308
      3      12      57
    660    2640   12540


In [37]:
! rm fence_amd.txt; touch fence_amd.txt
! ssh htseng@blissey "lscpu | grep 'Model name:'; cd ~/courses/CS203/demo/multiprocessor; for i in {1..10000}; do ./fence 2>> fence_amd.txt; done;"
! grep "(0, 0)" fence_amd.txt |wc
! grep "(0, 1)" fence_amd.txt |wc
! grep "(1, 1)" fence_amd.txt |wc
! grep "(1, 0)" fence_amd.txt |wc

Model name:                           AMD Ryzen 7 5700X 8-Core Processor
      3      12      57
   9992   39968  189848
      0       0       0
      5      20      95


### "mfence" instructions:

An instruction that forces all updates must finish before making progress after this instruction.

In [23]:
render_code("fence.c", show=["#ifdef MFENCE", "#endif"])

// fence.c:41-57 (17 lines)
#ifdef MFENCE
#define _update_var(_v, _x) { _v = (_x); asm("mfence"); }
void* modifya(void *z)
{
//  a=1;
  _update_var(a,1);
  x=b;
  return NULL;
}
void* modifyb(void *z)
{
//  b=1;
  _update_var(b,1);
  y=a;
  return NULL;
}
#endif

In [ ]:
!make mfence
!rm mfence.txt; touch mfence.txt
!for i in {1..10000}; do ./fence 2>> mfence.txt; done;
#!ssh htseng@gengar "cd ~/courses/CS203/demo/multiprocessor; for i in {1..3000}; do ./mfence 2>> mfence.txt; done;"

make: 'mfence' is up to date.


In [ ]:
! grep "(0, 0)" mfence.txt |wc
! grep "(0, 1)" mfence.txt |wc
! grep "(1, 1)" mfence.txt |wc
! grep "(1, 0)" mfence.txt |wc

In [61]:
! rm mfence_amd.txt; touch mfence_amd.txt
! ssh htseng@blissey "cd ~/courses/CS203/demo/multiprocessor; for i in {1..10000}; do ./mfence 2>> mfence_amd.txt; done;"
! grep "(0, 0)" mfence_amd.txt |wc
! grep "(0, 1)" mfence_amd.txt |wc
! grep "(1, 1)" mfence_amd.txt |wc
! grep "(1, 0)" mfence_amd.txt |wc

      0       0       0
   9978   39912  189582
     13      52     247
      9      36     171
